In [2]:
!pip install xmltodict
!pip install gspread==3.6.0

# Importación librerias


In [3]:
import requests
import pandas as pd
import xml.etree.ElementTree as et
import xmltodict


# Variables generales

In [4]:
BASEURL = "http://tarea-4.2021-1.tallerdeintegracion.cl/"
CODES = ["CHL", "DEU", "BRA", "USA", "CAN", "JPN"]
CHILDNODES = ["GHO", "COUNTRY", "SEX", "YEAR", "GHECAUSES", "AGEGROUP", "Display", "Numeric", "Low", "High"]
DEATH_I = ["Number of deaths", "Number of infant deaths", "Number of under-five deaths", "Mortality rate for 5-14 year-olds (probability of dying per 1000 children aged 5-14 years)",
               "Adult mortality rate (probability of dying between 15 and 60 years per 1000 population)", "Estimates of number of homicides", "Crude suicide rates (per 100 000 population)", 
              "Mortality rate attributed to unintentional poisoning (per 100 000 population)", "Number of deaths attributed to non-communicable diseases, by type of disease and sex", 
              "Estimated road traffic death rate (per 100 000 population)", "Estimated number of road traffic deaths"]
WEIGHT_I = ["Mean BMI (kg/m2) (age-standardized estimate)", "Mean BMI (kg/m2) (crude estimate)", "Prevalence of obesity among adults, BMI > 30 (age-standardized estimate) (%)",
            "Prevalence of obesity among children and adolescents, BMI > +2 standard deviations above the median (crude estimate) (%)",
            "Prevalence of overweight among adults, BMI > 25 (age-standardized estimate) (%)", "Prevalence of overweight among children and adolescents, BMI > +1 standard deviations above the median (crude estimate) (%)",
            "Prevalence of underweight among adults, BMI < 18.5 (age-standardized estimate) (%)", "Prevalence of thinness among children and adolescents, BMI < -2 standard deviations below the median (crude estimate) (%)"]
OTHERS_I = ["Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)", "Estimate of daily cigarette smoking prevalence (%)", "Estimate of daily tobacco smoking prevalence (%)",
  "Estimate of current cigarette smoking prevalence (%)", "Estimate of current tobacco smoking prevalence (%)", "Mean systolic blood pressure (crude estimate)", "Mean fasting blood glucose (mmol/l) (crude estimate)",
  "Mean Total Cholesterol (crude estimate)"]

INDICATORS = DEATH_I + WEIGHT_I + OTHERS_I
RARES = ["Mean BMI"] #indicadores que no estan textuales

# Funciones auxiliares

In [5]:
def clean_gho(gho):
  return gho.replace("&GreaterEqual;", ">").replace("&#xb2;", "2")

def filter_similar(gho):
  for i in RARES:
    if i in gho:
      return True
  return gho in INDICATORS

def get_xml_data(code):
  url = BASEURL + f"gho_{code}.xml"
  response = requests.get(url)
  dict_data = xmltodict.parse(response.content, encoding='utf-8')
  facts = dict_data['Data']['Fact']
  return facts

def clean_value(value):
  i = value.find("[")
  if i != -1:
    return value[:i]
  return value


# Extracción y limpieza de datos

In [27]:
frames = []
ghos = {}

for code in CODES:
  print("Extrayendo datos de", code, "...")
  ghos[code] = []
  facts = get_xml_data(code)
  df_cols = ["GHO", "COUNTRY", "SEX", "YEAR", "GHECAUSES", "AGEGROUP", "Display", "Numeric", "Low", "High"]
  rows = []
  
  for fact in facts: 
      row = {}
      gho = clean_gho(fact["GHO"])
      if filter_similar(gho):
        ghos[code].append(gho)
        row["GHO"] = gho
        for col in df_cols[1:]:
          try:
            value = fact[col]
            row[col] = clean_value(value)
          except:
            row[col] = ""
        rows.append(row)

  df= pd.DataFrame(rows, columns = df_cols)
  frames.append(df)
  #df.to_csv(f'{code}.csv', index=False, sep=";")

final_df = pd.concat(frames)
#final_df.to_csv('data.csv', index=False, sep=";")
final_df.head(200)





VOY CHL
VOY DEU
VOY BRA
VOY USA
VOY CAN
VOY JPN


,GHO,COUNTRY,SEX,YEAR,GHECAUSES,AGEGROUP,Display,Numeric,Low,High
0,Mean BMI (kg/m2) (crude estimate),Chile,Male,1980,,20+ years,24.3,24.30000,22.40000,26.20000
1,Mean BMI (kg/m2) (crude estimate),Chile,Male,1981,,20+ years,24.3,24.30000,22.60000,26.10000
2,Mean BMI (kg/m2) (crude estimate),Chile,Male,1982,,20+ years,24.4,24.40000,22.70000,26.00000
3,Mean BMI (kg/m2) (crude estimate),Chile,Male,1983,,20+ years,24.4,24.40000,22.90000,26.00000
4,Mean BMI (kg/m2) (crude estimate),Chile,Male,1997,,20+ years,25.8,25.80000,25.00000,26.60000
...,...,...,...,...,...,...,...,...,...,...
195,Mean BMI (kg/m2) (crude estimate),Chile,Female,2005,,20+ years,27.6,27.60000,26.70000,28.40000
196,Mean Total Cholesterol (crude estimate),Chile,Male,1981,,25+ years,5.0,5.00000,4.20000,5.80000
197,Mean Total Cholesterol (crude estimate),Chile,Male,1983,,25+ years,5.0,5.00000,4.30000,5.70000
198,Mean Total Cholesterol (crude estimate),Chile,Male,1990,,25+ years,5.0,5.00000,4.60000,5.40000


In [28]:
# cambiando . por , por formato de google sheets
final_df["Display"]=final_df["Display"].str.replace('.',',')
final_df["Numeric"]=final_df["Numeric"].str.replace('.',',')
final_df["Low"]=final_df["Low"].str.replace('.',',')
final_df["High"]=final_df["High"].str.replace('.',',')

final_df.head(10)
  

,GHO,COUNTRY,SEX,YEAR,GHECAUSES,AGEGROUP,Display,Numeric,Low,High
0,Mean BMI (kg/m2) (crude estimate),Chile,Male,1980,,20+ years,"24,3","24,30000","22,40000","26,20000"
1,Mean BMI (kg/m2) (crude estimate),Chile,Male,1981,,20+ years,"24,3","24,30000","22,60000","26,10000"
2,Mean BMI (kg/m2) (crude estimate),Chile,Male,1982,,20+ years,"24,4","24,40000","22,70000","26,00000"
3,Mean BMI (kg/m2) (crude estimate),Chile,Male,1983,,20+ years,"24,4","24,40000","22,90000","26,00000"
4,Mean BMI (kg/m2) (crude estimate),Chile,Male,1997,,20+ years,"25,8","25,80000","25,00000","26,60000"
5,Mean BMI (kg/m2) (crude estimate),Chile,Male,1998,,20+ years,"25,9","25,90000","25,20000","26,60000"
6,Mean BMI (kg/m2) (crude estimate),Chile,Male,1999,,20+ years,"26,0","26,00000","25,40000","26,70000"
7,Mean BMI (kg/m2) (crude estimate),Chile,Male,2000,,20+ years,"26,1","26,10000","25,50000","26,80000"
8,Mean BMI (kg/m2) (crude estimate),Chile,Female,1982,,20+ years,"24,1","24,10000","21,40000","26,70000"
9,Mean BMI (kg/m2) (crude estimate),Chile,Female,1983,,20+ years,"24,2","24,20000","21,70000","26,70000"


# Poblar google sheets

In [31]:
import gspread
# import os
# os.environ['PRIVATE_KEY_ID'] = "key_id"
# os.environ['PRIVATE_KEY']= "key"

# credenciales con variables de entorno
# credenciales = {
#   "type": "service_account",
#   "project_id": "tarea4-316723",
#   "private_key_id": os.environ['PRIVATE_KEY_ID'],
#   "private_key": os.environ['PRIVATE_KEY'],
#   "client_email": "tien-287@tarea4-316723.iam.gserviceaccount.com",
#   "client_id": "104130424307692461188",
#   "auth_uri": "https://accounts.google.com/o/oauth2/auth",
#   "token_uri": "https://oauth2.googleapis.com/token",
#   "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
#   "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/tien-287%40tarea4-316723.iam.gserviceaccount.com"
# }
 
# método mostrado en el enunciado
from gspread_dataframe import set_with_dataframe
# ACCES GOOGLE SHEET
gc = gspread.service_account(filename='credenciales.json')
sh = gc.open_by_key('1ASB1Ql8HozR7GcNm6hN_FO491IZRpIyO8lFjXkZEDK4')
worksheet = sh.get_worksheet(0) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, final_df)
